In [1]:
import tensorflow as tf

In [23]:
import numpy as np


In [24]:
from keras.applications.mobilenet import MobileNet

In [16]:
from keras.applications.mobilenet import MobileNet
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten, BatchNormalization
from keras.models import Sequential
def model():
    base_mobilenet_model = MobileNet(input_shape = (300,300,3), 
                                     include_top = False, 
                                     weights = None)
    base_model = Sequential()
    base_model.add(BatchNormalization(input_shape = (300,300,3)))
    base_model.add(base_mobilenet_model)
    base_model.add(BatchNormalization())
    base_model.add(GlobalAveragePooling2D())
    base_model.add(Dropout(0.5))
    base_model.add(Dense(2, activation = 'linear' )) # linear is what 16bit did
    base_model.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics=['accuracy'] )
    base_model.summary()
    return base_model

In [9]:
from keras.preprocessing.image import ImageDataGenerator
gen = ImageDataGenerator(rescale=1.0/255.0)
train_it=gen.flow_from_directory('V:\\V_Drive\\DATA\\liver_binary/train/',
		class_mode='categorical',classes=['abnormal','normal'], batch_size=5, target_size=(300, 300))

test_it = gen.flow_from_directory('V:\\V_Drive\\DATA\\liver_binary/test/',
		class_mode='categorical',classes=['abnormal','normal'], batch_size=5, target_size=(300, 300))

Found 136 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


In [17]:
modelf = model()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_7 (Batch (None, 300, 300, 3)       12        
_________________________________________________________________
mobilenet_1.00_300 (Model)   (None, 9, 9, 1024)        3228864   
_________________________________________________________________
batch_normalization_8 (Batch (None, 9, 9, 1024)        4096      
_________________________________________________________________
global_average_pooling2d_4 ( (None, 1024)              0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 2050      
Total params: 3,235,022
Trainable params: 3,211,080
Non-trainable params: 23,942
_______________________________________

In [19]:
history = modelf.fit_generator(train_it, steps_per_epoch=len(train_it),
		validation_data=test_it, validation_steps=len(test_it), epochs=5, verbose=0)

_, acc = modelf.evaluate_generator(test_it, steps=len(test_it), verbose=0)

print('> %.3f' % (acc * 100.0))

> 50.000


In [20]:
modelf.save('liver_mobile.h5')

In [21]:
import tensorflow as tf 
model=tf.keras.models.load_model('liver_mobile.h5')
converter=tf.lite.TFLiteConverter.from_keras_model(model)


In [22]:
tflite_model=converter.convert()
file=open('liver_mnet.tflite','wb')
file.write(tflite_model)

12816704

In [25]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
img=load_img('V:\\V_Drive\\DATA\\liver_binary\\binary_data_liver\\abnormal/grade2-001.jpg',target_size=(300, 300))
img=img_to_array(img)
img=img.reshape(1, 300, 300,3)
img = img.astype('float32')
img = img - [123.68, 116.779, 103.939]

In [ ]:
result = model.predict(img)
if(result[0][0]>result[0][1]):
    print("Abnormal")
else:
      print("normal")